In [1]:
import tensorflow as tf
import numpy as np
import time

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
with tf.name_scope('input'):
	x = tf.placeholder("float", shape=[None, 784])
	y_ = tf.placeholder("float", shape=[None, 10])

def weight_variable(shape):
	initial = tf.truncated_normal(shape, stddev=0.1);
	return tf.Variable(initial)

def conv2d(x, W, padding):
	return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding=padding)

def max_pool_2x2(x):
	return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2,1], padding='SAME')

#First Convolutional Layer
with tf.name_scope('1st_CNN'):
	W_conv1 = weight_variable([5, 5, 1, 6])
	x_image = tf.reshape(x, [-1,28,28,1])
	h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, padding='SAME')) #[28,28,32]
	h_pool1 = max_pool_2x2(h_conv1) #[14,14,6]

#Second Convolutional Layer
with tf.name_scope('2rd_CNN'):
	W_conv2 = weight_variable([5, 5, 6, 16])
	h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2, padding='VALID')) #[10,10,16]
	h_pool2 = max_pool_2x2(h_conv2) #[5,5,16]

#First Densely Connected Layer
with tf.name_scope('Densely_NN1'):
	W_fc1 = weight_variable([ 5* 5* 16, 120])
	h_pool2_flat = tf.reshape(h_pool2, [-1, 5* 5* 16])
	h_fc1= tf.nn.relu(tf.matmul(h_pool2_flat , W_fc1)) # [120]

#Second Densely Connected Layer
with tf.name_scope('Densely_NN2'):
	W_fc2 = weight_variable([ 120, 84])
	h_fc2= tf.nn.relu(tf.matmul(h_fc1 , W_fc2)) # [120]

#Readout Layer
with tf.name_scope('Softmax'):
	W_fc3 = weight_variable([84, 10])
	#h_fc2 = tf.matmul(h_fc1_drop, W_fc2)
	h_fc3 = tf.matmul(h_fc2, W_fc3)
	y_conv=tf.nn.softmax(h_fc3)

with tf.name_scope('Loss'):
	cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))

with tf.name_scope('Train'):
	train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
	#train_step = tf.train.AdamOptimizer(5e-5).minimize(cross_entropy)

with tf.name_scope('Accuracy'):
	correct_prediction = tf.equal(tf.argmax(y_conv ,1), tf.argmax(y_,1))
	accuracy = tf.reduce_mean(tf.cast(correct_prediction , "float"))

In [5]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
lista = [W_conv1, W_conv2, W_fc1, W_fc2, W_fc3]
listw = []

In [6]:
for i in range(5):
    exec('w{}=np.load("tfweight{}.npy")'.format(i,i))
    exec('listw.append(w{})'.format(i))

In [7]:
for (a,w) in zip(lista, listw):
    sess.run(tf.assign(a,w))

In [8]:
mnist.test.images.shape

(10000, 784)

In [9]:
mnist.test.labels.shape

(10000, 10)

In [10]:
starttime=time.time()
acc = accuracy.eval(feed_dict={x: mnist.test.images[200:201,:], y_: mnist.test.labels[200:201,:]})
#print("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images[200:201,:], y_: mnist.test.labels[200:201,:]}))
endtime=time.time()
print(acc)
print("run time=%s s"%(endtime-starttime))

1.0
run time=0.15305829048156738 s


In [11]:
total_time = 0
correct = 0
for i in range(10000):
    starttime=time.time()
    acc = accuracy.eval(feed_dict={x: mnist.test.images[i:i+1,:], y_: mnist.test.labels[i:i+1,:]})
    endtime=time.time()
    tmp_time = endtime - starttime
    total_time = total_time + tmp_time
    correct = correct + acc
print("total tensorflow run time=%s s"%(total_time))
print("correct number=%d"%correct)

total tensorflow run time=72.68461990356445 s
correct number=9915


In [13]:
total_time = 0
correct = 0
for i in range(2):
    starttime=time.time()
    acc = accuracy.eval(feed_dict={x: mnist.test.images[i*5000:(i+1)*5000,:], y_: mnist.test.labels[i*5000:(i+1)*5000,:]})
    endtime=time.time()
    tmp_time = endtime - starttime
    total_time = total_time + tmp_time
    correct = correct + acc*5000
print("once tensorflow run time=%s s"%(total_time))
print("correct number=%d"%correct)

once tensorflow run time=51.476648807525635 s
correct number=9915
